# Agents

In the last lab
- we looked at a basic agent structure and agent calling tools.
- we also got introduced to the different agentic patterns.

In this lab, we see concrete implementation of these patterns.

_Each module is typically dependent on the prior modules having been completed successfully_


As we go intot the Agents Module we need a new Python dependancy

In [1]:
%pip install openai-agents==0.0.13


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import openai
import re
import httpx
import os
import requests
import rich
import json
from openai import OpenAI
import requests
from agents import Agent, ModelSettings, function_tool, Runner, AsyncOpenAI, OpenAIChatCompletionsModel


os.environ["OPENAI_API_KEY"] = "dummy_key" 
api_key = "placeholder" 
base_url = "http://localhost:11434/v1/"
model = "llama3.2:3b-instruct-fp16" 

# Configure the model
model = OpenAIChatCompletionsModel( 
     model=model,
     openai_client=AsyncOpenAI(base_url=base_url, api_key=openai.api_key),
 )

from rich import print
print("[green] Model setup[/green]")

 Model setup

## Agents participating in a Trouble shooting Workflow
1. The investigation into four specialized “agents” that work together like a miniature on-call team. 
1. First, the Dependency Identifier agent figures out which backend services power the shopping cart. 
1. Next, the Change Tracker agent looks at those services to see what code or configuration changes went out recently. 
1. Then the Error Finder agent scans their logs to pull out any real-time error messages. 
1. Finally, the Debugger agent takes those three fact-sets—dependencies, changes, and errors—and reasons through them step by step to deliver a clear, human-friendly diagnosis of the most likely root cause.

_This will be agentic workflow will be improved further in the next lesson_

In [109]:
@function_tool
def get_dependency(service:str) ->list[str]:
    dep_service=["ProductCatalogService","CheckoutService","UserProfileService"]
    return dep_service

did_agent = Agent(
    name="DependencyIdentifier Agent",
    instructions=(
        "An incident will be passed on.\n"
        "From that, firstly identify the affected service name only.\n"
        "Next, identify what are the service dependencies for that service.\n"
        "Just return all service names in a comma separated format like a python list[str]. Also include the affected service.\n"
        "And nothing else"
    ),
    model= model,
    tools=[get_dependency],
    model_settings=ModelSettings(temperature=0.0),
)

#Useful for debugging individual agents
#dep_result = await Runner.run(did_agent, "Incident: Shopping cart response time has increased to 10 sec")
#print(dep_result.final_output)

print("[green] DependencyIdentifier Agent setup[/green]")

 DependencyIdentifier Agent setup

In [110]:
@function_tool
def get_changelog(service:list) ->list[str]:
    change_log=["ProductCatalogService changed","CheckoutService changed"]
    return change_log

change_agent = Agent(
    name="ChangeLog Agent",
    instructions=(
        "An array of service names will be passed on - like ['ProductCatalogService','CheckoutService','..'].\n"
        "Identify what has changed with these services and return them.\n"
        "Use available tools for this and do not generate data on our own.\n"
        "Just return all changes in a comma separated format like a python list[str].\n"
        "Do not return duplicate changes"
    ),
    model= model,
    tools=[get_changelog],
    model_settings=ModelSettings(temperature=0.0),
)
print("[green] ChangeLog Agent setup[/green]")

 ChangeLog Agent setup

In [8]:
@function_tool
def get_errorlog(service:list) ->list[str]:
    error_log=["ProductCatalogService is responding slowly"]
    return error_log

error_agent = Agent(
    name="Error Log Agent",
    instructions=(
        "An array of service names will be passed on - like ['ProductCatalogService','CheckoutService','..']. \n"
        "Note that all services may not have error messages and it is unlikely that same message appear in logs of all services. \n"
        "Use available tools for this and do not generate data on our own.\n"
        "The error messages will have service names in the messages. \n"
        "Identify which services have what kind of errors"
    ),
    model= model,
    tools=[get_errorlog],
    model_settings=ModelSettings(temperature=0.0),
)
print("[green] ErrorLog Agent setup[/green]")

 ErrorLog Agent setup

The Debugger Agent takes all the facts you’ve gathered and asks the LLM to reason about root cause. This does not use tool calling.



In [111]:
debugger_agent = Agent(
    name="Debugger Agent",
    instructions=(
        "You will be given:\n"
        "1. Incident details.\n"
        "2. Services that could have been root cause of the problem.\n"
        "3. Services that were changed in the time interval.\n"
        "4. Services that had errors in the logs.\n"
        "Based on the above, logically think through and conclude the most likely reason for this problem. \n"
        "Please lay down your thought process clearly that led you to the conclusion. "
    ),
    model= model,
    model_settings=ModelSettings(temperature=0.0),
)
print("[green] Debugger Agent setup[/green]")

 Debugger Agent setup

This is the orchestration function


In [112]:
import asyncio
async def orchestrate(input):
    # Call the intermediate agents to gather the facts
    # These all use tools heavily
    dep_result = await Runner.run(did_agent,input)
    change_result = await Runner.run(change_agent, dep_result.final_output)
    error_result = await Runner.run(error_agent, dep_result.final_output)

    services = dep_result.final_output               # e.g. ["foo","bar","baz"]
    changes  = change_result.final_output             # e.g. ["foo changed","bar changed"]
    errors   = error_result.final_output              # e.g. ["foo is responding slowly"]

    # Build a single prompt string:
    message = (
        "Incident details: " + input + "\n"
        "Affected services: " + services + "\n"
        "Changes detected: " + changes + "\n"
        "Error logs: " + errors + "\n"
        "Based on the above, logically think through and conclude the most likely reason for this problem. "
        "Please lay down your thought process clearly that led you to the conclusion."
    )
    print("\n")
    print("Input to the Deubgger Agent: ")
    print("-----------------------------")
    print(message)
    print("\n")
    # Invoke it:
    debugger_result = await Runner.run(debugger_agent, message)
    return debugger_result.final_output
print("[green] Agent orchestration setup[/green]")

 Agent orchestration setup

## Invoking the agentic workflow


In [113]:
input = "Incident: ShoppingCart  response time has increased to 10 sec"
diagnosis = await orchestrate(input)
print("=============================================")
print("=== Debugger Thought Process & Conclusion ===")
print("=============================================")
print(diagnosis)

Input to the Deubgger Agent:

-----------------------------

Incident details: Incident: ShoppingCart  response time has increased to 10 sec
Affected services: The affected service is ShoppingCart, and its dependencies are ProductCatalogService, 
CheckoutService, and UserProfileService.
Changes detected: The changes to the affected service 'ShoppingCart' are:

ProductCatalogService changed, 
CheckoutService changed
Error logs: {"name":"get_errorlog","parameters":{"service":["["}}
Based on the above, logically think through and conclude the most likely reason for this problem. Please lay down 
your thought process clearly that led you to the conclusion.

=============================================

=== Debugger Thought Process & Conclusion ===

=============================================

To determine the most likely reason for the increased response time of the ShoppingCart service, I'll analyze the 
given information step by step.

**Step 1: Identify the affected services**

The affected services are:

* ShoppingCart
* ProductCatalogService
* CheckoutService
* UserProfileService

These services are interconnected, and changes to one service can impact others. However, since the response time 
of the ShoppingCart service has increased significantly (10 seconds), it's likely that the issue is related to this
service.

**Step 2: Analyze the changes detected**

The changes detected in the time interval before the incident occurred are:

* ProductCatalogService changed
* CheckoutService changed

These changes suggest that there might be a ripple effect on the ShoppingCart service, as it depends on these 
services. However, we need to consider other factors as well.

**Step 3: Examine the error logs**

The error log provided is incomplete and doesn't provide much information about the specific issue. The log only 
mentions that an error occurred in the "get_errorlog" function, but it's unclear what this function does or how it 
relates to the ShoppingCart service.

However, since we're looking for a logical explanation, let's assume that the error logs might be related to one of
the services involved. Given the changes detected earlier, I'll focus on ProductCatalogService and CheckoutService 
as potential causes.

**Step 4: Consider possible connections between services**

Based on the dependencies between services:

* ShoppingCart depends on ProductCatalogService for product data.
* ShoppingCart depends on CheckoutService for payment processing.
* ProductCatalogService might be affected by changes in the database or data storage.
* CheckoutService might be affected by changes in payment gateways or payment processing algorithms.

Considering these connections, I'll propose a possible explanation:

**Most likely reason:**

The increased response time of the ShoppingCart service is likely due to issues with the ProductCatalogService. The
change to this service might have caused delays in retrieving product data, which is then used by the ShoppingCart 
service. This delay could be due to various reasons such as:

* Database or data storage issues
* Slow query performance
* Network connectivity problems

As a result, the ShoppingCart service is taking longer to process requests and respond with the expected results.

**Alternative explanation:**

Another possible explanation is that the changes to CheckoutService caused an issue in payment processing, which 
affected the ShoppingCart service. However, this would require more information about the specific error logs or 
changes made to CheckoutService to support this claim.

In conclusion, based on the given information and logical analysis, I believe that the most likely reason for the 
increased response time of the ShoppingCart service is related to issues with the ProductCatalogService.

# Microservices

There are several meaningful similarities between LLM-based AI agents and microservices:

## Similarities
#### Specialized functionality: 
Both are designed to handle specific tasks or domains. Microservices focus on particular business capabilities, while AI agents can be specialized for specific types of interactions or knowledge domains.
### Independent operation: 
Both can operate autonomously within their defined scope. Once configured, they can process requests without requiring constant supervision.
### Communication patterns: 
Both typically communicate via messages/APIs. Microservices use REST/gRPC/messaging protocols, while AI agents receive prompts and return responses through APIs.
### Composability: 
Both can be combined to build larger systems. Microservices can be orchestrated to create complex applications; similarly, multiple AI agents can work together in a workflow.
### Statelessness vs. statefulness: 
Basic implementations of both can be stateless, but more sophisticated versions maintain state. The Agent class you showed maintains conversation history, similar to how some microservices maintain session state.
### Scaling considerations: 
Both face similar operational challenges around scaling, monitoring, and versioning.

## Key differences:

### Implementation: 
Microservices are traditional code with deterministic logic, while LLM agents use probabilistic models. MCP Servers which expose tools to be used by Agents could be totally traditional code with deterministic logic.
### Predictability: 
Microservices have more predictable outputs for given inputs, while LLM responses can vary.


# AFTERWORD
Agents are an extremely powerful construct in the field of Generative AI:
1. You can achieve complex tasks designing appropriate agents and tools and driving interaction between the different agents.
1. There are known ways by which we can improve accuracy of the output. Much like human beings help check one another's work, agents can do the same.
1. External data retrieval and queries are carried out through the tools.
1. If agent processing needs to be vetted, make sure humans are used (human-in-the-loop) to are used to vet the agent output before it moves to the next step. Really, this is no different to how we operate in our real life with human beings - we have review and approval processes etc.